# Exposure Time Calculator

In this notebook we will be doing exposure time calculations for point sources, and extended sources with half-light radii of 0.2 arcsec or 0.3 arcsec.

When running this notebook you will be asked to select your filter, the Zodiacal light contribution, the S/N, the nature of your source, and the fit applied. 

The options for each are discussed below:

- Filters: F062, F087, F106, F129, F158, F184, F146, F213 
- Zodiacal light contributions (multiples of the minimum): 1.2, 1.4, 2.0, 3.5
- Source: point sources, objects with a half-light radius (HLR) = 0.2", objects with a HLR = 0.3"
    - Fit with a PSF (Point source only)
    - Fit with a 2 pixel circular aperture (Point source & HLR = 0.2")
    - Fit with a 3 pixel circular aperture 
    - Fit with a 4 pixel circular aperture
    - Fit with a 5 pixel circular aperture (HLR = 0.3" only)
    - Fit with a 6 pixel circular aperture (HLR = 0.2" & 0.3" only)
    
- S/N: 5, 10, 15, 20, 50

Exposure times are quantized in multiples of  3 readout frames, with the number of visits/dithers: 1

You can calculate either the magnitude for an object at a for a given exposure time and a S/N, or the exposure time needed for a given magnitude.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from scipy.interpolate import interp1d
import ipywidgets 
from ipywidgets import widgets

This notebook works by conducting a simple linear interpolation between the magnitude and exposure time for a given S/N.

# Select you Filter:

In [2]:
widg_flt = widgets.Dropdown(
    options=['F062', 'F087', 'F106', 'F129', 'F158', 'F184', 'F146', 'F213'],
    value='F062',
    description='Filter:',
    disabled=False,)

display(widg_flt)

Dropdown(description='Filter:', options=('F062', 'F087', 'F106', 'F129', 'F158', 'F184', 'F146', 'F213'), valu…

In [3]:
filter = widg_flt.value
print("You selected: "+filter)

You selected: F062


# Select the Zodiacal  light contribution:

In [4]:
widg_zod = widgets.Dropdown(
    options=[1.2, 1.4, 2.0, 3.5],
    value=1.2,
    description='Zodical light:',
    disabled=False,)

display(widg_zod)

Dropdown(description='Zodical light:', options=(1.2, 1.4, 2.0, 3.5), value=1.2)

In [5]:
zodi = widg_zod.value
print("You selected: "+str(zodi))

You selected: 1.2


# Select your kind of source and the fit to it 


In [6]:
widg_source = widgets.Dropdown(
    options=['point source: PSF', 'point source: CircAp = 2', 'point source: CircAp = 3', 'point source: CircAp = 4',
            'half-light radius=0.2": CircAp = 2', 'half-light radius=0.2": CircAp = 3', 'half-light radius=0.2": CircAp = 4', 'half-light radius=0.2": CircAp = 6',
            'half-light radius=0.3": CircAp = 3','half-light radius=0.3": CircAp = 4', 'half-light radius=0.3": CircAp = 5', 'half-light radius=0.3": CircAp = 6'],
    value='point source: PSF',
    description='Source:',
    disabled=False,)

display(widg_source)

Dropdown(description='Source:', options=('point source: PSF', 'point source: CircAp = 2', 'point source: CircA…

In [7]:
source = widg_source.value
print("You selected: "+source)

You selected: point source: PSF


# Select the S/N

In [8]:
widg_snr = widgets.Dropdown(
    options=[5, 10, 15, 20, 50],
    value=5,
    description='S/N:',
    disabled=False,)

display(widg_snr)

Dropdown(description='S/N:', options=(5, 10, 15, 20, 50), value=5)

In [9]:
snr = widg_snr.value
print("You selected: "+str(snr))

You selected: 5


# Read in the selected data file

By selecting a source you are reading in a certain data file. We will do this below using ascii.read.

In [10]:
ffn = ['point source: PSF', 'point source: CircAp = 2', 'point source: CircAp = 3', 'point source: CircAp = 4',
            'half-light radius=0.2": CircAp = 2', 'half-light radius=0.2": CircAp = 3', 'half-light radius=0.2": CircAp = 4', 'half-light radius=0.2": CircAp = 6',
            'half-light radius=0.3": CircAp = 3','half-light radius=0.3": CircAp = 4', 'half-light radius=0.3": CircAp = 5', 'half-light radius=0.3": CircAp = 6']
ffn2 = ['pts_PSFfit.txt', 'pts_CircAp_2.txt', 'pts_CircAp_3.txt', 'pts_CircAp_4.txt'
       'hlr2_CircAp_2.txt', 'hlr2_CircAp_3.txt', 'hlr2_CircAp_4.txt', 'hlr2_CircAp_6.txt',
       'hlr3_CircAp_3.txt','hlr3_CircAp_4.txt', 'hlr3_CircAp_5.txt', 'hlr3_CircAp_6.txt']

#PS files
if source==ffn[0]:
    data = ascii.read(ffn2[0])
if source==ffn[1]:
    data = ascii.read(ffn2[1])
if source==ffn[2]:
    data = ascii.read(ffn2[2])
if source==ffn[3]:
    data = ascii.read(ffn2[3])

#Half-light radius 0.2
if source==ffn[4]:
    data = ascii.read(ffn2[4])
if source==ffn[5]:
    data = ascii.read(ffn2[5])
if source==ffn[6]:
    data = ascii.read(ffn2[6])
if source==ffn[7]:
    data = ascii.read(ffn2[7])

#Half-light radius 0.3
if source==ffn[8]:
    data = ascii.read(ffn2[8])
if source==ffn[9]:
    data = ascii.read(ffn2[9])
if source==ffn[10]:
    data = ascii.read(ffn2[10])
if source==ffn[11]:
    data = ascii.read(ffn2[11])
 

Now we obtain the key values from the file

In [11]:
fl = np.array(data['Filt'], dtype=str)
zd = np.array(data['zodi'], dtype=float)
sn = np.array(data['SNR'], dtype=float)
mag = np.array(data['Mag_AB'], dtype=float)
exp = np.array(data['time'], dtype=float)

Next we sort based on what you the user input 

In [12]:
#Filter selection
a = np.where(fl==filter)
fa, za, sa, ma, ea = fl[a], zd[a], sn[a], mag[a], exp[a]

#Zodical light selection
b = np.where(za==float(zodi))
fb, zb, sb, mb, eb = fa[b], za[b], sa[b], ma[b], ea[b]

#S/N selection
c = np.where(sb==float(snr))
fc, zc, sc, mc, ec = fb[c], zb[c], sb[c], mb[c], eb[c]

tmin = np.min(ec)
tmax = np.max(ec)

Then we set up the interpolation, one based on magnitude and one on exposure time.

In [13]:
fm = interp1d(ec, mc) #Return a magnitude
ft = interp1d(mc, ec) #Return a time

# To calculate the magnitude of your object given an exposure time

Note that you can only specify times between the minimim and maximum values as given below. If a -9 is returned. this is because the time specified is outside of the given range. 

In [14]:
print("Minimum allowed time (sec): "+str(tmin))
print("Maximum allowed time (sec): "+str(tmax))

Minimum allowed time (sec): 10.0
Maximum allowed time (sec): 28902.0


In [15]:
widg_timein = widgets.FloatText(
    value=10,
    min= tmin,
    max = tmax,
    description='Exp time (s):',
    disabled=False
)

display(widg_timein)

FloatText(value=10.0, description='Exp time (s):')

In [16]:
print("Calculated Mag (AB): "+str(fm(widg_timein.value)))

Calculated Mag (AB): 20.9


# To calculate the exposure time needed for your given object

Note that you can only specify magnitudes between 20 and 30 AB.

In [17]:
widg_magin = widgets.FloatText(
    value=20,
    min= 20,
    max = 30,
    description='Exp time (s):',
    disabled=False
)

display(widg_magin)

FloatText(value=20.0, description='Exp time (s):')

In [18]:
print("Calculated Time (sec): "+str(ft(widg_magin.value)))

Calculated Time (sec): 10.0
